In [131]:
# Import required libraries
# pandas: for data manipulation and analysis
# requests: for making HTTP requests to Binance API
# math: for mathematical operations
# Decimal: for precise decimal arithmetic
import pandas as pd
import requests
import math
from decimal import Decimal

In [132]:
# Define the trading pair symbol and API endpoint
symbol = "SOLUSDT"  # Trading pair: Solana/USDT
url = "https://api.binance.com/api/v1/depth"  # Binance orderbook endpoint

# Set the interval for price level aggregation (0.1 USDT)
interval = Decimal('0.1')

# Parameters for the API request
# limit=1000 gets the top 1000 bids and asks
params = {
    "symbol": symbol,
    "limit": 1000,
}

In [133]:
data = requests.get(url, params).json()
print(data)

{'lastUpdateId': 25313466359, 'bids': [['189.05000000', '1.82800000'], ['189.04000000', '16.91600000'], ['189.03000000', '3.68100000'], ['189.02000000', '59.40300000'], ['189.01000000', '59.49700000'], ['189.00000000', '113.88800000'], ['188.99000000', '83.46000000'], ['188.98000000', '92.24300000'], ['188.97000000', '115.22800000'], ['188.96000000', '190.55500000'], ['188.95000000', '305.19900000'], ['188.94000000', '140.32000000'], ['188.93000000', '293.46300000'], ['188.92000000', '230.15100000'], ['188.91000000', '141.15100000'], ['188.90000000', '340.36900000'], ['188.89000000', '277.98800000'], ['188.88000000', '77.66200000'], ['188.87000000', '201.22900000'], ['188.86000000', '681.77400000'], ['188.85000000', '359.90000000'], ['188.84000000', '150.61600000'], ['188.83000000', '257.42700000'], ['188.82000000', '166.35900000'], ['188.81000000', '253.41500000'], ['188.80000000', '146.73300000'], ['188.79000000', '93.32500000'], ['188.78000000', '73.35400000'], ['188.77000000', '138

## Aggregating Bids Levels

In [134]:
# Create a DataFrame for bid orders
# Convert the bids data from the API response into a structured DataFrame
# Each row contains: price level and quantity available at that price
bids_levels = pd.DataFrame(data['bids'], columns=['price', 'quantity'], dtype=float)
bids_levels['side'] = 'bid'  # Add a column to identify this as bid orders
print(bids_levels)
print(bids_levels.dtypes)  # Display data types of each column

      price  quantity side
0    189.05     1.828  bid
1    189.04    16.916  bid
2    189.03     3.681  bid
3    189.02    59.403  bid
4    189.01    59.497  bid
..      ...       ...  ...
995  179.09     2.935  bid
996  179.08    21.871  bid
997  179.07     3.392  bid
998  179.06     1.507  bid
999  179.05    15.932  bid

[1000 rows x 3 columns]
price       float64
quantity    float64
side         object
dtype: object


In [135]:
# Calculate the minimum bid price level
# This rounds down the minimum bid price to the nearest interval (0.1 USDT)
min_bid_level = math.floor(min(bids_levels['price']) / float(interval)) * interval
min_bid_level

Decimal('179.0')

In [136]:
# Calculate the maximum bid price level
# Round up to the next interval and add one more interval for binning
max_bid_level = (math.ceil(max(bids_levels['price']) / float(interval)) + 1) * interval
max_bid_level

Decimal('189.2')

In [137]:
# Create an array of price levels for binning
# Generate price boundaries from min to max bid level with the specified interval
bid_levels_bounds = [float(min_bid_level + interval * i)
    for i in range(int((max_bid_level - min_bid_level) / interval) + 1)
]
bid_levels_bounds

[179.0,
 179.1,
 179.2,
 179.3,
 179.4,
 179.5,
 179.6,
 179.7,
 179.8,
 179.9,
 180.0,
 180.1,
 180.2,
 180.3,
 180.4,
 180.5,
 180.6,
 180.7,
 180.8,
 180.9,
 181.0,
 181.1,
 181.2,
 181.3,
 181.4,
 181.5,
 181.6,
 181.7,
 181.8,
 181.9,
 182.0,
 182.1,
 182.2,
 182.3,
 182.4,
 182.5,
 182.6,
 182.7,
 182.8,
 182.9,
 183.0,
 183.1,
 183.2,
 183.3,
 183.4,
 183.5,
 183.6,
 183.7,
 183.8,
 183.9,
 184.0,
 184.1,
 184.2,
 184.3,
 184.4,
 184.5,
 184.6,
 184.7,
 184.8,
 184.9,
 185.0,
 185.1,
 185.2,
 185.3,
 185.4,
 185.5,
 185.6,
 185.7,
 185.8,
 185.9,
 186.0,
 186.1,
 186.2,
 186.3,
 186.4,
 186.5,
 186.6,
 186.7,
 186.8,
 186.9,
 187.0,
 187.1,
 187.2,
 187.3,
 187.4,
 187.5,
 187.6,
 187.7,
 187.8,
 187.9,
 188.0,
 188.1,
 188.2,
 188.3,
 188.4,
 188.5,
 188.6,
 188.7,
 188.8,
 188.9,
 189.0,
 189.1,
 189.2]

In [138]:
# Assign each bid price to its corresponding price bin
# right=False means intervals are left-inclusive
# precision=10 ensures we don't lose decimal precision
bids_levels["bin"] = pd.cut(
    bids_levels['price'],
    bins=bid_levels_bounds,
    right=False,
    precision=10
)
bids_levels

,price,quantity,side,bin
0,189.05,1.828,bid,"[189.0, 189.1)"
1,189.04,16.916,bid,"[189.0, 189.1)"
2,189.03,3.681,bid,"[189.0, 189.1)"
3,189.02,59.403,bid,"[189.0, 189.1)"
4,189.01,59.497,bid,"[189.0, 189.1)"
...,...,...,...,...
995,179.09,2.935,bid,"[179.0, 179.1)"
996,179.08,21.871,bid,"[179.0, 179.1)"
997,179.07,3.392,bid,"[179.0, 179.1)"
998,179.06,1.507,bid,"[179.0, 179.1)"


In [139]:
# Group the bids by their price bins and aggregate the data
# Sum up quantities for each price level
# Keep the 'side' information (all 'bid' in this case)
# Reset index to make the bin column a regular column
bids_levels = bids_levels.groupby("bin").agg(
    quantity = ("quantity", "sum"),
    side = ("side", "first")
).reset_index()

# Extract the lower bound of each bin as the price level label
bids_levels['label'] = bids_levels['bin'].apply(lambda x: x.left)
bids_levels

C:\Users\Saksham Maggu\AppData\Local\Temp\ipykernel_2052\1285173725.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bids_levels = bids_levels.groupby("bin").agg(


,bin,quantity,side,label
0,"[179.0, 179.1)",45.637,bid,179.0
1,"[179.1, 179.2)",206.533,bid,179.1
2,"[179.2, 179.3)",295.957,bid,179.2
3,"[179.3, 179.4)",33.668,bid,179.3
4,"[179.4, 179.5)",50.033,bid,179.4
...,...,...,...,...
97,"[188.7, 188.8)",2455.549,bid,188.7
98,"[188.8, 188.9)",2573.103,bid,188.8
99,"[188.9, 189.0)",1932.139,bid,188.9
100,"[189.0, 189.1)",255.213,bid,189.0


## Aggregating Asks Levels

In [140]:
# Create a DataFrame for ask orders
# Convert the asks data from the API response into a structured DataFrame
# Each row contains: price level and quantity available at that price
asks_levels = pd.DataFrame(data['asks'], columns=['price', 'quantity'], dtype=float)
asks_levels['side'] = 'ask'  # Add a column to identify this as ask orders
print(asks_levels)
print(asks_levels.dtypes)  # Display data types of each column

      price  quantity side
0    189.06   373.475  ask
1    189.07   364.665  ask
2    189.08   405.349  ask
3    189.09   394.588  ask
4    189.10   226.081  ask
..      ...       ...  ...
995  199.01    22.996  ask
996  199.02   160.171  ask
997  199.03     2.346  ask
998  199.04     8.750  ask
999  199.05     1.650  ask

[1000 rows x 3 columns]
price       float64
quantity    float64
side         object
dtype: object


In [141]:
# Calculate minimum ask level
# Round down to nearest interval and subtract one interval to ensure all asks are included
min_ask_level = (math.floor(min(asks_levels['price']) / float(interval)) - 1) * interval
min_ask_level

Decimal('188.9')

In [142]:
# Calculate maximum ask level
# Round up to next interval and add one more interval for complete binning
max_ask_level = (math.ceil(max(asks_levels['price']) / float(interval)) + 1) * interval
max_ask_level

Decimal('199.2')

In [143]:
# Create an array of price levels for ask binning
# Generate price boundaries from min to max ask level with the specified interval
ask_levels_bounds = [float(min_ask_level + interval * i)
    for i in range(int((max_ask_level - min_ask_level) / interval) + 1)
]
ask_levels_bounds

[188.9,
 189.0,
 189.1,
 189.2,
 189.3,
 189.4,
 189.5,
 189.6,
 189.7,
 189.8,
 189.9,
 190.0,
 190.1,
 190.2,
 190.3,
 190.4,
 190.5,
 190.6,
 190.7,
 190.8,
 190.9,
 191.0,
 191.1,
 191.2,
 191.3,
 191.4,
 191.5,
 191.6,
 191.7,
 191.8,
 191.9,
 192.0,
 192.1,
 192.2,
 192.3,
 192.4,
 192.5,
 192.6,
 192.7,
 192.8,
 192.9,
 193.0,
 193.1,
 193.2,
 193.3,
 193.4,
 193.5,
 193.6,
 193.7,
 193.8,
 193.9,
 194.0,
 194.1,
 194.2,
 194.3,
 194.4,
 194.5,
 194.6,
 194.7,
 194.8,
 194.9,
 195.0,
 195.1,
 195.2,
 195.3,
 195.4,
 195.5,
 195.6,
 195.7,
 195.8,
 195.9,
 196.0,
 196.1,
 196.2,
 196.3,
 196.4,
 196.5,
 196.6,
 196.7,
 196.8,
 196.9,
 197.0,
 197.1,
 197.2,
 197.3,
 197.4,
 197.5,
 197.6,
 197.7,
 197.8,
 197.9,
 198.0,
 198.1,
 198.2,
 198.3,
 198.4,
 198.5,
 198.6,
 198.7,
 198.8,
 198.9,
 199.0,
 199.1,
 199.2]

In [144]:
# Assign each ask price to its corresponding price bin
# right=True means intervals are right-inclusive (opposite of bids)
# precision=10 ensures we don't lose decimal precision
asks_levels["bin"] = pd.cut(
    asks_levels['price'],
    bins=ask_levels_bounds,
    right=True,
    precision=10
)
asks_levels

,price,quantity,side,bin
0,189.06,373.475,ask,"(189.0, 189.1]"
1,189.07,364.665,ask,"(189.0, 189.1]"
2,189.08,405.349,ask,"(189.0, 189.1]"
3,189.09,394.588,ask,"(189.0, 189.1]"
4,189.10,226.081,ask,"(189.0, 189.1]"
...,...,...,...,...
995,199.01,22.996,ask,"(199.0, 199.1]"
996,199.02,160.171,ask,"(199.0, 199.1]"
997,199.03,2.346,ask,"(199.0, 199.1]"
998,199.04,8.750,ask,"(199.0, 199.1]"


In [145]:
# Group the asks by their price bins and aggregate the data
# Similar to bids processing:
# - Sum quantities for each price level
# - Keep the 'side' information
# - Reset index for regular column access
asks_levels = asks_levels.groupby("bin").agg(
    quantity = ("quantity", "sum"),
    side = ("side", "first")
).reset_index()

# Extract the lower bound of each bin as the price level label
asks_levels['label'] = asks_levels['bin'].apply(lambda x: x.left)
asks_levels

C:\Users\Saksham Maggu\AppData\Local\Temp\ipykernel_2052\3123010510.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  asks_levels = asks_levels.groupby("bin").agg(


,bin,quantity,side,label
0,"(188.9, 189.0]",0.000,None,188.9
1,"(189.0, 189.1]",1764.158,ask,189.0
2,"(189.1, 189.2]",3584.958,ask,189.1
3,"(189.2, 189.3]",2731.249,ask,189.2
4,"(189.3, 189.4]",3332.048,ask,189.3
...,...,...,...,...
98,"(198.7, 198.8]",274.013,ask,198.7
99,"(198.8, 198.9]",458.158,ask,198.8
100,"(198.9, 199.0]",4079.938,ask,198.9
101,"(199.0, 199.1]",195.913,ask,199.0


## Concatening both bid and ask in orderbook

In [146]:
# Combine both bid and ask levels into a single orderbook DataFrame
# 1. Concatenate asks and bids
# 2. Remove any price levels with zero quantity
# 3. Sort by price (label) in descending order to show highest price first
orderbook = pd.concat([asks_levels, bids_levels])
orderbook = orderbook[orderbook['quantity'] > 0]
orderbook = orderbook.sort_values(by='label', ascending=False)
print(orderbook.to_string())

                bin   quantity side  label
101  (199.0, 199.1]    195.913  ask  199.0
100  (198.9, 199.0]   4079.938  ask  198.9
99   (198.8, 198.9]    458.158  ask  198.8
98   (198.7, 198.8]    274.013  ask  198.7
97   (198.6, 198.7]    176.613  ask  198.6
96   (198.5, 198.6]    528.993  ask  198.5
95   (198.4, 198.5]    367.481  ask  198.4
94   (198.3, 198.4]   5612.997  ask  198.3
93   (198.2, 198.3]    191.339  ask  198.2
92   (198.1, 198.2]    385.853  ask  198.1
91   (198.0, 198.1]    145.361  ask  198.0
90   (197.9, 198.0]   3059.972  ask  197.9
89   (197.8, 197.9]    225.032  ask  197.8
88   (197.7, 197.8]    329.619  ask  197.7
87   (197.6, 197.7]    111.440  ask  197.6
86   (197.5, 197.6]    103.676  ask  197.5
85   (197.4, 197.5]    138.187  ask  197.4
84   (197.3, 197.4]    308.471  ask  197.3
83   (197.2, 197.3]    112.139  ask  197.2
82   (197.1, 197.2]    499.926  ask  197.1
81   (197.0, 197.1]     84.650  ask  197.0
80   (196.9, 197.0]   2785.791  ask  196.9
79   (196.8